In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sample_dist = 0.3
aggregation_range = 15
eigen_threshold = 0.05
angle_threshold = 0.12
radius = 1.5

In [3]:
point_cloud_location = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

In [4]:
# Preprocess the point cloud
pcd = util.preProcessCloud(pcd)
pcd_points = np.asarray(pcd.points)

# Detect planes, intersections, and anchor points
segment_models, segments, segment_indices, main_surface_idx = util.multiOrderRansac(pcd, pt_to_plane_dist=0.4)
angles_rad = util.findAnglesBetweenPlanes(segment_models, main_surface_idx)
intersection_lines = util.findIntersectionLines(segment_models, main_surface_idx)
anchor_points = util.findAnchorPoints(segment_models, segments, intersection_lines, main_surface_idx)

bend_length_calculator = BendLengthCalculator(pcd, anchor_points, intersection_lines, eigen_threshold, angle_threshold, aggregation_range, sample_dist, radius)
bend_edges = bend_length_calculator.compute_bend_lengths()

KeyboardInterrupt: 

In [ ]:
all_normals, pointwise_variance = util.calculatePointwiseNormalVariance(pcd, radius=1.5)
core_indices = util.getCorePoints(pointwise_variance)
clusters = util.growRegionsAroundIntersections(anchor_points, core_indices, pointwise_variance, pcd_points, bend_edges, variance_percentile=95)

In [ ]:
cluster_derivatives = {}
for cluster_id, cluster in clusters.items():
    cluster_indices = np.array([idx for idx in cluster])
    points = pcd_points[cluster_indices]
    normals = all_normals[cluster_indices]
    cluster_derivatives[cluster_id] = util.calculate_normal_derivatives(normals, points, radius=2, verbose=False)

In [ ]:
class ArcLengthCalculator:
    def __init__(self, pcd, intersection_line, bend_edges, bend_cluster_points, cluster_derivatives, step_size=0.5):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)
        self.intersection_line = intersection_line
        self.bend_cluster_points = bend_cluster_points
        self.bend_cluster_derivatives = cluster_derivatives
        self.bend_cluster_kdtree = cKDTree(self.bend_cluster_points)
        self.pcd_kdtree = cKDTree(self.points)
        self.start_point, self.end_point = map(np.array, bend_edges)
        self.step_size = step_size
        self.it = 0

        self.pcd.paint_uniform_color([0.6, 0.6, 0.6])
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window("ArcLengthCalculator")
        self.vis.add_geometry(self.pcd)

        self.vis.register_key_callback(262, self.sample_point)
        self.sample_point(self.vis)

    def sample_point(self, vis):
        if hasattr(self, 'line_set'):
            self.vis.remove_geometry(self.line_set)

        sampled_point = self.start_point + (self.it*self.step_size)*self.intersection_line[0]/np.linalg.norm(self.intersection_line[0])
        if np.linalg.norm(sampled_point - self.start_point) > np.linalg.norm(self.end_point - self.start_point):
            return
        
        idx = self.bend_cluster_kdtree.query(sampled_point)[1]
        base = self.bend_cluster_points[idx]
        neighbor_indices = self.bend_cluster_kdtree.query_ball_point(base, radius)
        average_direction = np.mean(self.bend_cluster_derivatives[neighbor_indices], axis=0)
        average_direction /= np.linalg.norm(average_direction)
        end = base + average_direction * 2

        self.line_set = self.create_arrow_line(base, end, color=[1, 0, 0])

        self.vis.add_geometry(self.line_set)
        self.it += 1

    # def next_neighborhood(self, vis):
    #     """ Move to the next neighborhood when right arrow key is pressed. """
    #     self.current_index = (self.current_index + 500) % len(self.pcd.points)
    #     self._update_neighborhood()

    def create_arrow_line(self, start, end, color):
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start, end])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        line_set.colors = o3d.utility.Vector3dVector([color])
        return line_set

    def run(self):
        self.vis.run()
        self.vis.destroy_window()

In [ ]:
bend_cluster_indices = np.array([idx for idx in clusters[1]])
bend_cluster_points = pcd_points[cluster_indices]

print(f'length of cluster points: {len(bend_cluster_points)}')
print(f'length of derivatives: {len(cluster_derivatives[1])}')

length of cluster points: 10422
length of derivatives: 10422


In [ ]:
visualizer = ArcLengthCalculator(pcd, intersection_lines[1], bend_edges[1], bend_cluster_points, cluster_derivatives[1])
visualizer.run()

[Open3D INFO]   -- Mouse view control --
[Open3D INFO]     Left button + drag         : Rotate.
[Open3D INFO]     Ctrl + left button + drag  : Translate.
[Open3D INFO]     Wheel button + drag        : Translate.
[Open3D INFO]     Shift + left button + drag : Roll.
[Open3D INFO]     Wheel                      : Zoom in/out.
[Open3D INFO] 
[Open3D INFO]   -- Keyboard view control --
[Open3D INFO]     [/]          : Increase/decrease field of view.
[Open3D INFO]     R            : Reset view point.
[Open3D INFO]     Ctrl/Cmd + C : Copy current view status into the clipboard.
[Open3D INFO]     Ctrl/Cmd + V : Paste view status from clipboard.
[Open3D INFO] 
[Open3D INFO]   -- General control --
[Open3D INFO]     Q, Esc       : Exit window.
[Open3D INFO]     H            : Print help message.
[Open3D INFO]     P, PrtScn    : Take a screen capture.
[Open3D INFO]     D            : Take a depth capture.
[Open3D INFO]     O            : Take a capture of current rendering settings.
[Open3D INFO

In [ ]:
import pickle

with open("bend_visualization_data.pkl", "wb") as f:
    pickle.dump({
        "intersection_line": intersection_lines[1],
        "bend_edges": bend_edges[1],
        "cluster_points": bend_cluster_points,
        "cluster_derivatives": cluster_derivatives[1],
        "all_normals": all_normals,
        "segment_indices": segment_indices
    }, f)